# Проект 1. Предобработка данных

Проект выполнил: студент Казачинский Андрей

## Обзор данных

Для проведения анализа импортируем необходимые библиотеки для анализа данных и поместим исходный датасет в тип данных Data Frame

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

df = pd.read_csv('/datasets/data.csv', encoding = 'utf-8')

Выведем первые десять строк исходного дата сета для получения предварительного представления о данных:

In [2]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


По этим данным мы имеем 5 столбцов со строковыми числами данных и 6 с числовыми в разных типах (float&int)

Но что с пропусками?

In [4]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В столбцах `days_employed` & `total_income` имеются пропуски, которые подлежат обработке 

**Обзор: числовые столбцы**

Просмотрим минимальное и максимальное значение в столбцах `days_employed` и `total_income`

In [5]:
range_job_days=df['days_employed'].max() - df['days_employed'].min()
range_job_days/365

1151.0804119887628

74 года трудового стажа - это слишком много и не годится для анализа. Нужно будет заменить отрицательные значения

In [6]:
df['total_income'].max() - df['total_income'].min()

2244936.7649294725

Разброс значений получился достаточно большой

In [7]:
df['dob_years'].min()

0

Даже младенцы знают, что такое кредит... Это тоже придется исправлять

In [8]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

По основным числовым столбцам (не являющихся идентификаторами) получили, что почти все они нуждаются в обработке своих значения для приведения к нормальному виду. Столбец `days_employed` содержит большое количество данных, которые не являются адекватными реальности, более того, данные о количестве дней занятости не являются значимыми при оценке кредитоспособности - учитываться не будут

**Обзор данных строковых столбцов**

In [9]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [10]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [11]:
df['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [12]:
df['purpose'].value_counts(ascending=False)

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Получили, что столбцы `education`, `purpose` нуждаются в допополнительной обработке, чтобы привести их к виду, способному к обработке. Остальные столбцы имеют приемлемый вид.
Остальные строковые столбцы имеют надлежащий вид.

В исходном наборе данных 11 столбцов с различными данными о клиентами. Среди них можно выделить несколько групп переменных:

1) Ключевые параметры: `debt` (индикатор надежности заемщиков), `total_income` (ежемесячный доход), `purpose`(цель кредита)
2) Независимые переменные, но влияющие на принятие решений: `children`, `education`, `family_status` и связанные с ними идентификаторы

**В рамках обзора удалось получить предварительное представление о данных клиентов, подавших заявку на получение кредита**

 ## Предобработка данных
 
 Заполнение значений в столбце пропущенных значений в столбце `total_income`
 
 Выведем значение медианы для последующей сверки значений

In [13]:
old_median = df['total_income'].median()
old_median

145017.93753253992

Созданим словарь по типу:

Тип занятости(`income_type`): значение медианы для этой группы(`median_value`)

In [14]:
income_type = df['income_type'].unique()

income_dict = {key: df.loc[df['income_type'] == key, 'total_income'].median() for key in income_type} 
income_dict

{'сотрудник': 142594.39684740017,
 'пенсионер': 118514.48641164352,
 'компаньон': 172357.95096577113,
 'госслужащий': 150447.9352830068,
 'безработный': 131339.7516762103,
 'предприниматель': 499163.1449470857,
 'студент': 98201.62531401133,
 'в декрете': 53829.13072905995}

In [15]:
try:
    for key in income_dict:
        df.loc[(df['income_type'] == key) & (df['total_income'].isnull() == True), 'total_income'] = income_dict[key]
except:
    print('Repeat logical indexation')


In [16]:
new_median = df['total_income'].median()
new_median

142594.39684740017

Значение медианного дохода по столбцу изменилось - оно стало меньше из-за неравномерности пропусков. Наверняка в кредите нуждались пенсионеры, студенты и люди в декрете, которые забыли проставить свой доход.

In [17]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Замену нулевых значений предлагается произвести с помощью меры среднего значения, так как в распределении возраста не должно происходить серьезных выбросов.

In [18]:
mean_age = int(df['dob_years'].mean())
mean_age

43

In [19]:
df['dob_years']= df['dob_years'].fillna(mean_age)

In [20]:
df['dob_years'].min()

0

<div class="alert alert-block alert-danger">
ЗАполнять пропуски довольно опасное занятие, т.к. всегда сказывается на итоговый результат. Поэтому нам надо максимально аккуратно и точно подбирать способ заполнение. И я думаю обобщенное усреденение - не этот случай. Можно попытаться сегментировать метрику среднего по другим параметрам - так будет точнее и логичнее
</div>
    
---

Теперь минимальный возраст в данных заемщиков соответствует общепринятому в большинстве стран возрасту совершеннолетия

In [21]:
df[df.isnull()].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Пропуски устранены
Далее предлагается произвести замену отрицательных значений.
Есть два пути как это сделать:

**1) Применить функцию abs() из стандартных библиотек Python**

**2) Домножить отрицательные значений на -1, предварительно отобрав их с помощью метода loc и логической индексации**

Мы выбрали 2-ой путь:

In [22]:
df.loc[df['days_employed'] < 0, 'days_employed'] = df['days_employed']*(-1)
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Теперь нормализуем значение по столбцу. Основная суть: по законам РФ можно работать, начиная с 14-го возраста, соответствено, трудовой стаж в днях не может превышать значения по формуле: (Возраст клиента - 14 лет)*365 дней.

Таким образом, если значение превышает данный показатель - приравняем получившееся значение в стаж.

In [23]:
def normalize_work_days(row):
    dob_days = row['days_employed']
    max_days = (row['dob_years'] - 14)*365.25
    if dob_days >= max_days: 
        return max_days
    else: 
        return dob_days

Удостоверимся, что наша функция работает:

In [24]:
try:
    df['days_employed'] = df.apply(normalize_work_days, axis=1)
    print(df['days_employed'].max()/365)
except:
    print('Что-то пошло не так')

60.04109589041096


Чисто теоретически возможно работать около 60 лет. Отсортируем по наибольшим значениям и взглянем на значения, где наибольший трудовой стаж:

In [25]:
df.sort_values(by='days_employed', ascending = False).head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4895,0,21915.00,74,высшее,0,женат / замужем,0,F,пенсионер,0,134935.354225,покупка своего жилья
19642,0,21915.00,74,среднее,1,вдовец / вдова,2,F,пенсионер,0,45089.546419,приобретение автомобиля
2557,0,21915.00,74,среднее,1,женат / замужем,0,F,пенсионер,0,42927.300898,автомобили
3460,0,21915.00,74,среднее,1,женат / замужем,0,M,пенсионер,0,54754.745517,операции со своей недвижимостью
20610,0,21549.75,73,среднее,1,вдовец / вдова,2,F,пенсионер,0,64437.765650,покупка жилой недвижимости
6406,0,21549.75,73,среднее,1,женат / замужем,0,F,пенсионер,0,119190.487878,покупка жилья
229,0,21549.75,73,среднее,1,вдовец / вдова,2,F,пенсионер,0,136897.008375,сделка с подержанным автомобилем
19385,0,21549.75,73,начальное,3,гражданский брак,1,F,пенсионер,0,100166.624237,на проведение свадьбы
1826,0,21549.75,73,Среднее,1,вдовец / вдова,2,F,пенсионер,0,74284.473064,покупка жилой недвижимости
5804,0,21549.75,73,среднее,1,женат / замужем,0,F,пенсионер,0,126603.199013,покупка коммерческой недвижимости


In [26]:
# А сколько будет 60 лет в днях?
(74-14)*365

21900

Так, функция сработала верно. Несмотря на то, что дальше этот столбец не понадобится для ответа на вопросы, но лучше иметь обработанный столбец со значениями, которые близки к реальности.

И последнее: посмотрим как стало выглядеться среднее значение по столбцу

In [27]:
days_employed_mean = df['days_employed'].mean()
days_employed_mean

4839.9987620322645

Заполним пропущенные значения в этом столбце и пойдем к следующим столбцам:

In [28]:
df['days_employed'] = df['days_employed'].fillna(days_employed_mean)

Пропуски в данном датафрейме устранены

In [29]:
df.sort_values(by='dob_years')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6859,1,-5113.500000,0,неоконченное высшее,2,гражданский брак,1,F,сотрудник,0,128246.646591,покупка недвижимости
16042,2,-5113.500000,0,среднее,1,женат / замужем,0,F,сотрудник,0,137563.630693,жилье
7034,0,-5113.500000,0,высшее,0,Не женат / не замужем,4,F,пенсионер,0,263121.074528,образование
21179,2,-5113.500000,0,высшее,0,женат / замужем,0,M,компаньон,0,240702.007382,строительство жилой недвижимости
4147,0,-5113.500000,0,среднее,1,в разводе,3,M,сотрудник,0,108130.933212,покупка жилой недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
3460,0,21915.000000,74,среднее,1,женат / замужем,0,M,пенсионер,0,54754.745517,операции со своей недвижимостью
4895,0,21915.000000,74,высшее,0,женат / замужем,0,F,пенсионер,0,134935.354225,покупка своего жилья
19642,0,21915.000000,74,среднее,1,вдовец / вдова,2,F,пенсионер,0,45089.546419,приобретение автомобиля
11532,0,6682.867814,74,среднее,1,гражданский брак,1,F,сотрудник,0,98945.906177,сыграть свадьбу


Пустой фрейм - все пропуски устранены

**Обработка столбца `children`**

Как мы выяснили, у нас есть аномальные значения (-1, 20). Первое значение, могло получиться в ходе сбоя в системе - единицу записали как (-1). Исправим это через функцию abs(). Значение 20 - могло получиться в результате прибавления ноля в строке записи - разделим на 10.

In [30]:
def normalize_children(row):
    value = row['children']
    if value == 20 : return (value // 10)
    else: return abs(value)

In [31]:
try:
    df['children'] = df.apply(normalize_children, axis=1)
except:
    print('Что-то пошло не так')

In [32]:
df['children'].value_counts(ascending=False)

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Значения нормализованы и стали похожи на реальные анкеты пользователей.

Теперь заменим вещественные значения в графах дохода и дней занятости с помощью метода astype()

In [33]:
df[['days_employed', 'total_income']] = df[['days_employed', 'total_income']].astype('int')

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Приведение к целочисленному типу произведено успешно

Теперь настала очередь разобраться с дублированными данными

**Обработка текстовых данных**

Начнем обработку с понижения регистра текстовых данных в графе `education` при помощи метода lower() для работы со строками

In [35]:
df['education'] = df['education'].str.lower()
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Метод сработал безупречно

Поиск дублированных данных будет производиться по исходному df c параметром keep=False для метода .duplicated, чтобы лучше понять данные:

In [36]:
df.loc[df.duplicated(keep=False) == True].sort_values(by=['dob_years'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
20297,1,4839,23,среднее,1,гражданский брак,1,F,сотрудник,0,142594,сыграть свадьбу
8853,1,4839,23,среднее,1,гражданский брак,1,F,сотрудник,0,142594,сыграть свадьбу
15892,0,4839,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,142594,сделка с подержанным автомобилем
19321,0,4839,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,142594,сделка с подержанным автомобилем
3452,0,4839,29,высшее,0,женат / замужем,0,M,сотрудник,0,142594,покупка жилой недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
5865,0,4839,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,118514,операции со своей недвижимостью
9528,0,4839,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,118514,операции со своей недвижимостью
9604,0,4839,71,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
7938,0,4839,71,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы


В исходном датасете обнаружились дубликаты. Удалим их с помощью метода drop_duplicates() и сбросим индекс

In [37]:
df = df.drop_duplicates().reset_index(drop = True)

In [38]:
df.loc[df.duplicated(keep=False) == True]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Дубликаты удалены

Следующий этап - лемматизация значений графы "Цель кредита" для её дальнейшей категоризации

In [39]:
m = Mystem()

In [40]:
def lemma(row):
    text = row['purpose']
    lemmas = ' '.join(m.lemmatize(text))
    return lemmas

In [41]:
df['purpose_lemmed'] = df.apply(lemma, axis=1)

In [42]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmed
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье \n
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль \n
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье \n
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование \n
4,0,14244,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба \n


In [43]:
df['purpose_lemmed'].value_counts()

автомобиль \n                                    972
свадьба \n                                       791
на   проведение   свадьба \n                     768
сыграть   свадьба \n                             765
операция   с   недвижимость \n                   675
покупка   коммерческий   недвижимость \n         661
операция   с   жилье \n                          652
покупка   жилье   для   сдача \n                 651
операция   с   коммерческий   недвижимость \n    650
покупка   жилье \n                               646
жилье \n                                         646
покупка   жилье   для   семья \n                 638
строительство   собственный   недвижимость \n    635
недвижимость \n                                  633
операция   со   свой   недвижимость \n           627
строительство   жилой   недвижимость \n          624
покупка   недвижимость \n                        621
покупка   свой   жилье \n                        620
строительство   недвижимость \n               

**Категоризация данных**

Нами были получили лематизированный список слов в  столбцах. Дальнейший шаг их классификация на основе полученных уникальных значений.

Будет выделено 5 категорий:

1) Автомобиль

2) Жилье

3) Недвижимость (покупка жилой площади с коммерческой целью)

4) Образование

5) Свадьба

In [44]:
def uni_categories(row):
    words = row['purpose_lemmed']
    if 'образование' in words:
        return 'образование'
    elif ('недвижимость' or 'сдача') in words:
        return 'недвижимость'
    elif 'жилье' in words:
        return 'жилье'
    elif 'автомобиль' in words:
        return 'автомобиль'
    elif 'свадьба'in words:
        return 'свадьба'

In [45]:
df['purpose_unique'] = df.apply(uni_categories, axis=1)

In [46]:
df['purpose_unique'].value_counts()

недвижимость    6351
жилье           4460
автомобиль      4306
образование     4013
свадьба         2324
Name: purpose_unique, dtype: int64

In [47]:
df['purpose_unique'].isnull().sum()

0

Замена на категориальные слова произведена успешна

In [48]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmed,purpose_unique
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье \n,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль \n,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье \n,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование \n,образование
4,0,14244,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба \n,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,покупка жилье \n,жилье
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,операция с жилье \n,жилье
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование \n,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,на проведение свадьба \n,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,покупка жилье для семья \n,жилье


Удалим промежуточный столбец с леммами, оставив исходный с данными о целях кредита. 

In [49]:
df = df.drop('purpose_lemmed', 1) 

Категоризация остальных данных будет производиться  по переменным возраста заемщика (`dob_years`), суммарного дохода за период (`total_income`) и наличию детей(`children`).

Для этого напишем несколько вспомогательных функций

In [50]:
def children_status(row):
    kid_num = row['children']
    if kid_num == 0:
        return 'childless'
    elif kid_num > 2:
        return 'large family'
    else:
        return 'with children'

In [51]:
df['children_cat']=df.apply(children_status, axis=1)

In [52]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_unique,children_cat
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье,with children
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,with children
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье,childless
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,large family
4,0,14244,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,childless


Мы категоризовали данные по количеству детей - многодетная семья, бездетный, с детьми

Следующий этап - **категоризация доходов**. Мы разделим заемщиков на 4 группы:

1) low(низкий), для которых значение `total_income` меньше 1-го квартиля

2) middle low (средний низкий), для которых значение `total_income` больше 1-го квартиля, но меньше медианы

3) middle high (средний высокий), для которых значение `total_income` больше 2-го квартиля, но меньше 3-го

4) high (высокий), с самыми высокими показателями

Для этого найдем значения квантилей 1/4, 1/2, 3/4 распределения, напишем функцию и применим её к датафрейму, данные категории сохраним в столбец `income_cat`

In [53]:
income_q1 = df['total_income'].quantile(0.25)
income_q2 = df['total_income'].median()
income_q3 = df['total_income'].quantile(0.75)
print(f'Значение первого квартиля: {income_q1}\nЗначение медианы: {income_q2}\nЗначение третьего квартиля: {income_q3}')

Значение первого квартиля: 107623.0
Значение медианы: 142594.0
Значение третьего квартиля: 195820.25


In [54]:
def income_categorize(row):
    income = row['total_income']
    if income < income_q1:
        return 'low'
    elif (income_q1 <= income) and (income < income_q2):
        return 'middle_low'
    elif (income_q2 <= income) and (income < income_q3):
        return 'middle_high'
    else:
        return 'high'

In [55]:
df['income_cat'] = df.apply(income_categorize, axis = 1)

In [56]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_unique,children_cat,income_cat
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье,with children,high
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,with children,middle_low
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье,childless,middle_high
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,large family,high
4,0,14244,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,childless,middle_high
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,жилье,childless,high
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,жилье,childless,high
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,childless,middle_low
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,with children,low
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,жилье,childless,middle_high


И последняя категоризация будет произведена по семейному положению. Здесь мы предлагаем выделить следующие категории:

1) Married

2) Partnership

3) Single

Это нужно для того, чтобы свернуть некоторые категории и объединить данные. Напомним уникальные значения для столбца family:

In [57]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [58]:
family_dict = {
    'женат / замужем': 'married',
    'гражданский брак':'partnership',
    'вдовец / вдова':'single',
    'в разводе':'single',
    'Не женат / не замужем':'single'
}

In [59]:
df['family_status'] = df['family_status'].map(family_dict)

In [60]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_unique,children_cat,income_cat
0,1,8437,42,высшее,0,married,0,F,сотрудник,0,253875,покупка жилья,жилье,with children,high
1,1,4024,36,среднее,1,married,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,with children,middle_low
2,0,5623,33,среднее,1,married,0,M,сотрудник,0,145885,покупка жилья,жилье,childless,middle_high
3,3,4124,32,среднее,1,married,0,M,сотрудник,0,267628,дополнительное образование,образование,large family,high
4,0,14244,53,среднее,1,partnership,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,childless,middle_high
5,0,926,27,высшее,0,partnership,1,M,компаньон,0,255763,покупка жилья,жилье,childless,high
6,0,2879,43,высшее,0,married,0,F,компаньон,0,240525,операции с жильем,жилье,childless,high
7,0,152,50,среднее,1,married,0,M,сотрудник,0,135823,образование,образование,childless,middle_low
8,2,6929,35,высшее,0,partnership,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,with children,low
9,0,2188,41,среднее,1,married,0,M,сотрудник,0,144425,покупка жилья для семьи,жилье,childless,middle_high


## Анализ данных

На данном этапе мы построим сводные таблицы по техническому заданию отдела, чтобы выделить закономерности между различными показателями и тем, как они влияют на возврат кредита вовремя.

**Есть ли зависимость между наличием детей и возвратом кредита в срок?**

Используем метод pivot_table для построения сводной таблицы и функции mean (для получения доли должников), sum (количества должников), count(подсчета ответов). 

In [61]:
stat_children = df.pivot_table(index='children_cat', values='debt', aggfunc=['mean','sum','count'])

In [62]:
stat_children.columns = ['Доля должников, %','Должников в категории, чел','Кол-во анкет']
stat_children['Доля должников, %'] = (stat_children['Доля должников, %']*100).round(2)

In [63]:
stat_children.sort_values(by='Доля должников, %', ascending=True)

,"Доля должников, %","Должников в категории, чел",Кол-во анкет
children_cat,,,
childless,7.54,1063,14091
large family,8.16,31,380
with children,9.27,647,6983


По данной таблице видно, что в каждой категории доля должников не превышает 10%. Наименьшее - 7,54% в группе бездетных, следом в больших семьях (от 3-х детей) и показатель 9,27% у группы c 1-2 ребенком у заемщика.

По эти данным мы не можем сказать, что существует значимая связь - объем категорий распределился слишком неравномерно (самая маленькая имеет только 380 ответов).

**Исходя из этого, мы заявляем, что возврат кредита в срок строго не зависит от наличия и количества детей у заемщика**

Однако, мы должны заметить, что в данной выборке лучше всего показали себя бездетные. Значит, требуются дальнейшие сравнительные исследования

**Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [64]:
stat_family = df.pivot_table(index='family_status', values='debt', aggfunc=['mean','sum','count'])

In [65]:
stat_family.columns = ['Доля должников, %','Должников в категории, чел','Кол-во анкет']
stat_family['Доля должников, %'] = (stat_family['Доля должников, %']*100).round(2)

In [66]:
stat_family.sort_values(by='Доля должников, %', ascending=True)

,"Доля должников, %","Должников в категории, чел",Кол-во анкет
family_status,,,
married,7.55,931,12339
single,8.50,422,4964
partnership,9.35,388,4151


По данной таблице видно, что в каждой категории доля должников не превышает 10%. Наименьшее - 7,55% в группе женатых, следом в группе гражданского брака (partnership) и показатель оказался равен 9,35% у группы одиноких.

Данные группы оказываются более соотносимы между собой. Группа "partnership" показала себя лучше на 0,75%, а отличие состоящих в браке от одиночек составило 1,8%.

***Исходя из этого, мы заявляем, что женатые люди в среднем чаще возвращают кредит в срок***

Однако, мы должны заметить, что величина разности не является такой уж критической требует дальнейшего экспериментального подтверждения для установления воспроизводимости результатов на разных выборках.

**Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [67]:
stat_income = df.pivot_table(index='income_cat', values='debt', aggfunc=['mean','sum','count'])

In [68]:
stat_income.columns = ['Доля должников, %','Должников в категории, чел','Кол-во анкет']
stat_income['Доля должников, %'] = (stat_income['Доля должников, %']*100).round(2)

In [69]:
stat_income.sort_values(by='Доля должников, %', ascending=True)

,"Доля должников, %","Должников в категории, чел",Кол-во анкет
income_cat,,,
high,7.14,383,5364
low,7.96,427,5364
middle_high,8.64,546,6317
middle_low,8.73,385,4409


По данной таблице видно, что категории дохода примерно соотносимы между собой. 
1) Лица с высоким доходом чаще возвращают кредит в срок. Показатель - 7,14% должников

2) Лица с низким доходом идут следом с показателем 7,96%

3) Группа со средним доходом показазала себя хуже остальных, значения в них практически одинаковые 8,64%, 8,73%, как для тех, кто ближе к низкому доходу, так и для тех, кто обладает большим достатком.

***Таким образом, доля возвратов кредита в срок изменяется нелинейно, группы с низким и высоким доходом похожи друг на друга, а группа со средним доходом чаще задерживает выплаты***

**Как разные цели кредита влияют на его возврат в срок?**

In [70]:
stat_purpose = df.pivot_table(index='purpose_unique', values='debt', aggfunc=['mean','sum','count'])

In [71]:
stat_purpose.columns = ['Доля должников, %','Должников в категории, чел','Кол-во анкет']
stat_purpose['Доля должников, %'] = (stat_purpose['Доля должников, %']*100).round(2)

In [72]:
stat_purpose.sort_values(by='Доля должников, %', ascending=True)

,"Доля должников, %","Должников в категории, чел",Кол-во анкет
purpose_unique,,,
жилье,6.91,308,4460
недвижимость,7.46,474,6351
свадьба,8.00,186,2324
образование,9.22,370,4013
автомобиль,9.36,403,4306


По данным сводной таблицы о целях кредита различия уже более ощутимы:

1) Группа тех, кто берет ипотеку (как для личных, так и для коммерческих целей), является самой надежной. Процент должников среди тех, кто берет ипотеку для личных целей - 6,91%, для коммерческих - 7,46%.

2) Группа тех, кто берет кредит для проведения свадьбы, также имеет низкий показатель - 8% ровно.

3) Хуже всего показали себя те, кто берет кредит для целей образования (9,22%) и для покупки автомобиля или операций с ТС (9,36%)

Учитывая это, стоит отметить, что цель ипотеки является более надежной для выдачи кредита банком. Скорее всего, люди, которые решаются на покупку жилья четко знают свои финансовые возможности и могут не выполнять обязательства из-за "форс-мажорных обстоятельств". 

***Автомобильные кредиты и кредиты на образования являются более опасными, как по данным о возврате кредита в срок, так и по своей сути.***

Инвестиции в образование имеют долгосрочную перспективу и могут не сразу окупиться после обучения из-за переменчивого характера трудового рынка и невозможности точно предсказать спрос на нем после завершения обучения.

Автомобильное направление является опасным при выдаче кредита сразу по нескольким основаниям:

Во-первых, это может быть сговор с целью поделить деньги между заемщиком и держателем авто. Здесь необходимо дополнительно привлекать оценщиков

Во-вторых, люди берущие кредит на автомобиль - не могут купить его сами и не захотели накопить средства для его покупки. Положение таких людей в финансовом плане также будет являться скорее всего неустойчивым. Сформируем таблицу, что оценить кредит по целям и уровню доходу одновременно:

In [73]:
stat_purpose_income = df.groupby(['purpose_unique', 'income_cat']).agg({'debt':['mean','sum','count']})

In [74]:
stat_purpose_income.columns = ['Доля должников, %','Должников в категории, чел','Кол-во анкет']
stat_purpose_income['Доля должников, %'] = (stat_purpose_income['Доля должников, %']*100).round(2)

In [75]:
stat_purpose_income.sort_values(by='Доля должников, %').head()

Доля должников, %  Должников в категории, чел  \
purpose_unique income_cat                                                   
свадьба        high                      5.03                          29   
жилье          high                      5.75                          66   
               low                       6.33                          71   
недвижимость   low                       7.08                         110   
жилье          middle_high               7.35                          98   

                            Кол-во анкет  
purpose_unique income_cat                 
свадьба        high                  576  
жилье          high                 1147  
               low                  1122  
недвижимость   low                  1554  
жилье          middle_high          1333

In [76]:
stat_purpose_income.sort_values(by='Доля должников, %').tail()

Доля должников, %  Должников в категории, чел  \
purpose_unique income_cat                                                   
свадьба        middle_high               9.25                          62   
образование    middle_high               9.65                         117   
автомобиль     low                       9.67                         101   
               middle_high              10.11                         124   
образование    middle_low               10.69                          87   

                            Кол-во анкет  
purpose_unique income_cat                 
свадьба        middle_high           670  
образование    middle_high          1213  
автомобиль     low                  1044  
               middle_high          1227  
образование    middle_low            814

По данным такого кросс-сравнения, мы можем дать более детальный ответ.

Самая надежная категория заемщиков - люди с высоким доходом с целями свадьбы, покупки собственного жилья. Следом - люди с низким доходом также с ипотекой. Замыкают пятерку - люди со средне-высоким доходом также по ипотеке.

Самая неблагонадежная категория - люди со средним доходом, которые берут кредит на образование и автомобили, а также с низким доходом для покупки собственного авто.

***Данная таблица и ей подобные могут быть очень полезны при оценке индивидуальных заявок в спорных ситуациях принятия решения.***

## Выводы

Нами была проделана работа по оценке исходных данных, их фильтрации, обработке с применением знаний и методов полученных в ходе обучения по программе и, в частности, курса "Предобработка данных".

По обработанным данным был произведен анализ с помощью агрегированния данных и построения сводных таблиц. По результатам такого анализа, мы обнаружили, что:
1) связь между количеством детей и возвратом кредита в срок является либо слабой, либо нерепрезентативной (из-за разности количества анкет в каждой из групп);

2) лучше возвращают кредит люди, состоящие в браке или гражданском партнерстве;

3) группы с высоким и низким доходом чаще возвращают кредит в срок, чем в группе со средним доходом;

4) существует связь между целями кредита и благонадежностью заемщика.

***В целом, полученные сведения могут быть потенциально полезны заказчику при построении системы кредитного скоринга.***

